<a href="https://colab.research.google.com/github/9mak/Langchain_GithubLoader/blob/main/Langchain_GithubLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧑‍💼README
- author: [9mak](https://github.com/9mak/zenn)
- created_at: 2024-10-19

### 実行方法
1. OpenAIのAPIキーを発行
2. 「ランタイム > すべてのセルを実行」
3. OpenAI APIキーをInputFormに入力してEnter

### 注意
- **OpenAIのAPI利用料金がかかります**
　ちょっと使っただけでもお金かかるので本当に気を付けてください
- 学習させたいリポジトリは`# GitHubリポジトリの設定`以下の値を変更してください
- 質問は`# 質問の設定と回答の取得`以下の`query`の値を変更してください

In [ ]:
import os
from getpass import getpass

# OpenAI APIキーの設定
os.environ["OPENAI_API_KEY"] = getpass("OpenAI APIキーを入力してください: ")

In [ ]:
# 必要なパッケージのインストール
!pip install langchain==0.3.4
!pip install langchain-community==0.3.3
!pip install langchain-openai==0.2.3
!pip install openai==1.52.0
!pip install chromadb==0.5.15
!pip install GitPython==3.1.43

In [ ]:
import os
import shutil
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.document_loaders import GitLoader
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# OpenAI APIキーを環境変数から取得
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY環境変数が設定されていません。")

# GitHubリポジトリの設定
clone_url = "https://github.com/9mak/9mak"
branch = "main"
repo_path = "./temp_repo/"
filter_ext = ".md"

try:
    if os.path.exists(repo_path):
      clone_url = None

    # GitLoaderの設定
    loader = GitLoader(
        clone_url=clone_url,
        branch=branch,
        repo_path=repo_path,
        file_filter=lambda file_path: file_path.endswith(filter_ext),
    )

    # インデックスの作成
    index = VectorstoreIndexCreator(
        vectorstore_cls=Chroma,
        embedding=OpenAIEmbeddings(api_key=api_key),
    ).from_loaders([loader])

    print("インデックスの作成が完了しました。")

    # 質問の設定と回答の取得
    query = "GithubのProfileにどんなものが設定されていますか？"
    print(f"質問: {query}")

    # RetrievalQAチェーンの作成
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(api_key=api_key),
        retriever=index.vectorstore.as_retriever(),
        return_source_documents=True
    )

    # 質問の実行
    result = qa_chain.invoke({"query": query})
    print(f"回答: {result['result']}")

except Exception as e:
    print(f"エラーが発生しました: {e}")